In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
from dotenv import load_dotenv
import os

# Load variables from .env
load_dotenv()

# Get the keys
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")


In [83]:
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [84]:
%pwd

'c:\\Projects\\Rag-Chatbot'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Projects\\Rag-Chatbot'

In [86]:
extracted_data = load_pdf_file(data='Data/')

In [87]:
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return doc

In [88]:
documents=chunk_data(docs=extracted_data)
documents[:3]

[Document(metadata={'producer': 'pdfTeX-1.40.18', 'creator': "LaTeX with 'moderncv' package", 'creationdate': '2023-04-04T05:04:00+00:00', 'author': 'Monisha Jegadeesan gray', 'keywords': 'Monisha Jegadeesan gray , curriculum vitæ, resumé', 'moddate': '2023-04-04T05:04:00+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017) kpathsea version 6.2.3', 'subject': 'Resumé of Monisha Jegadeesan gray', 'title': 'Monisha Jegadeesan gray  –  Software Engineer, Google', 'trapped': '/False', 'source': 'Data\\cv.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='Monisha Jegadeesan\nSoftware Engineer, Google\nH +91 9035212894\nB monishaj.65@gmail.com\nÍ monisha-jega.github.io\nmonisha-jega\nmonisha-jegadeesan\nEducation\n2015-2020 Dual Degree (B.Tech + M.Tech) in Computer Science and Engineering\nIndian Institute of Technology Madras, Chennai, India CGPA: 8.78\n2015 XII - Karnataka Board,KLE Society’s Independent PU College, Bangalore 97.30 %\

continue from here

In [89]:

text = []
id = 1
for doc in documents:
  page_number = doc.metadata['page']
  content = doc.page_content
  text.append(
      {
          "_id":"rec"+str(id),
          "chunk_text": content
      }
  )
  id+=1

In [65]:
text = convert_doc_to_list(documents=documents)

In [90]:
len(text)

14

In [6]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=pinecone_api_key)

In [7]:
index_name = "man"

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"llama-text-embed-v2",
            "field_map":{"text": "chunk_text"}
        }
    )

In [13]:
index = pc.Index(host="https://man-uzat91r.svc.aped-4627-b74a.pinecone.io")


c:\Projects\Rag-Chatbot\rag_chatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
x = 0
y = 96
for i in range(len(text)//96 + 1):
  index.upsert_records(
      "example-namespace",
      text[x:y]
  )
  x += 96
  y += 96

In [ ]:

from langchain.schema import Document

def retrieve_query(query, k=4):
    matching_results = index.search(
        namespace="example-namespace",
        query={
            "inputs": {"text": query},
            "top_k": k
        },
        fields=["chunk_text"]
    )

    hits = matching_results['result']['hits']

    documents = []
    for hit in hits:
        fields = hit.get('fields', {})
        text = fields.get('chunk_text', '')
        metadata = {
            "id": hit.get('_id'),
            "score": hit.get('_score')
        }
        documents.append(Document(page_content=text, metadata=metadata))
    return documents


In [9]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI

In [10]:
llm=ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.5)
chain=load_qa_chain(llm, chain_type="stuff")

C:\Users\whoma\AppData\Local\Temp\ipykernel_23700\2741113786.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.5)
C:\Users\whoma\AppData\Local\Temp\ipykernel_23700\2741113786.py:2: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/ma

In [27]:
def retrieve_answers(query):
    doc_search = retrieve_query(query)

    response = chain.invoke({"input_documents": doc_search, "question": query})
    return doc_search
    """response["output_text"]"""


In [36]:
our_query = "tell me about monisha's teaching experience"
answer = retrieve_answers(our_query)
answer[1].metadata["score"]

4 documents retrieved


0.2440079003572464